# Asyncio

<br>

<figure>
    <img src="./images/StansKitchen.jpg" width="80%">
    <figcaption style="text-align: center">
        <a href="https://www.wattpad.com/748141112-22-short-stories-from-gravity-falls-story-10">
            <em>https://www.wattpad.com/748141112-22-short-stories-from-gravity-falls-story-10</em>.
        </a>
    </figcaption>
</figure>

# Contents<a id="contents"></a>

* [CPU & I/O Bound Programs](#cpu-and-io-bound)
* [Deeper in Concurrency: Coroutines vs. Threading vs. Parallelism](#deeper-in-concurrency)
* [Asyncio](#asyncio)
  * [Sync, or "How Normal People Do It"](#sync)
  * [Async, But... ("Old-Way Acync")](#async-but)
  * [Kinda Async, or "Looks Like Asyncio but Doesn't Quack Like Asyncio"](#kinda-async)
  * [Real Async, or "Finally Something Strange is Starting to Happen"](#real-async)
  * [Task, or Something That Is Not Ready Yet](#task)
* ["Real-World" Example \#1: Books](#books)
* ["Real-World" Example \#2: Butterbrot](#butterbrot)
* [References](#references)

## CPU & I/O Bound Programs<a id="cpu-and-io-bound"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Зачем нужны корутины?
Корутины — как способ асинхронно выполнять ряд действий.

Вообще, при написании кода (даже до написания) надо задумываться о том
1. Будет ли код ожидать чего-то, например данных из базы данных или от сервера. Если ответ "да", то производительность кода, скорее всего, ограничена операциями ввода-вывода (I/O bound code).

<figure>
    <img alt="Load Bar" src="https://media.giphy.com/media/WqA2ZMPc1ddTIbRupZ/giphy.gif" width="50%">
    <figcaption style="text-align: center">
        <a href="https://giphy.com/gifs/gay-loading-WqA2ZMPc1ddTIbRupZ">
            <em>https://giphy.com/gifs/gay-loading-WqA2ZMPc1ddTIbRupZ</em>.
        </a>
    </figcaption>
</figure>

2. Будет ли код выполнять какие-то сложные долгие вычисления? Если ответ "да", то, скорее всего, производительность кода ограничена количеством ядер процессора (CPU bound code).

```python
for i in range(10000000000000):
    _ = 1 + 1
```

Ввиду такого свойства текущей реализации CPython языка Питон, как [GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock), CPU bound программы никак не получится оптимизировать, кроме как использовать библиотеку `multiprocessing` для параллельного выполнения кода на нескольких ядрах (которая просто запускает по отдельному интерпретатору Питона на каждом ядре, на физическом ядре). 
Что же до I/O bound программ, то тут на помощь как раз могут прийти корутины (и не только).

## Deeper in Concurrency: Coroutines vs. Threading vs. Parallelism<a id="deeper-in-concurrency"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

В Питоне есть несколько способов *одновременно* выполнять несколько вещей:
* нити (потоки)
* процессы
* задачи (можно думать о задачах в рамках конкретной библиотеки `asyncio`; основано на корутинах)

В целом подходы могут показаться похожими, но если копать глубже, то у каждого есть свои уникальные черты и область применимости.
Так, потоки отличаются от задач порядком выполнения: остановка и продолжение потоков регулируется операционной системой, в то время как в `asyncio` задачи кооперируют друг с другом: каждая может давать другим понять, что она готова продолжать работу.
Ещё преимущество `asyncio` перед потоками в том, что при выполнении корутин дополнительные расходы по памяти меньше.
При самом создании корутины никакого исполнения вообще не происходит: объект корутины может быть передан окрестратору (Event Loop), который уже может запустить корутину на выполнение тотчас же или позже.

Далее мы подробнее обсудим библиотеку `asyncio`.

## Asyncio<a id="asyncio"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Библиотека `asyncio` появилась в Питоне ~3.4.
Постоянно дополнялась, обновлялась (некоторые удобные абстракции работают только начиная с Питона 3.7).

Плюсы `asyncio` по сравнению с чистыми корутинами:
* С помощью специального синтаксиса корутины выделяются в языке более явно: так, это уже не просто "расширенная версия геренатора".
* Есть готовая реализация Event Loop, который, как планировщик задач или дирижёр, вызывает те корутины-задачи, которые уже готовы продолжать работу. Event Loop ждёт, пока не произойдёт какое-либо событие (что-то скачалось, данные подгрузились, какая-то задача завершилась, ...), и реагирует по ситуации.

![](./images/event_loop.png)
*Tasks в Event Loop (https://docs.python.org/3.4/library/asyncio-task.html).*

* Есть возможность использовать асинхронность в контекстных менеджерах и при итерациях в цикле `for`

Корутина, начиная с Питона ~3.4, может определаться так
```python
async def read_data(database):
    pass
```

Ключевое здесь — `async def`.
Модификатор `async` делает функцию асинхронной и позволяет использовать внутри такой (и только такой) функции конструкцию с `await`:
```python
async def read_data(database):
    data = await database.fetch('SELECT name FROM mushrooms WHERE cap_color = "RED";')
    
    # Doint something with the data...
```

При этом `await` служит аналогом `yield from`.
И после `await` также может идти не что угодно, а специальный объект (Awaitable объект, который реализует специальный magic метод).
При использовании `await` вне `async def` функций будет ошибка (`SyntaxError`).
Именно `await` передаёт контроль из корутины вызывающему корутину коду и позволяет переключаться на выполнение других задач, пока текущая задача под `await` не будет выполнена.
При определении корутины с помощью `async def` использовать `yield` внутри уже нельзя: как способы возвращения значения вызывающему коду могут использоваться только `return` и `await`.

Корутины в рамках `asyncio` — как задачи (`Task`), которые регулирует Event Loop.
При этом часто для задач можно встретить обозначение `Future`: что-то, что ещё не готово, но посчитается когда-то в будущем (класс `Task` наследуется от класса `Future`).
Метод для создания задачи `create_task` появился позже, чем метод для создания фьючи `create_future` (в Питоне 3.7).

Хоть код читается как "последовательность действий", как и в случае с просто корутинами, но, благодаря орекстратору Event Loop, который решает, какая задача будет выполняться следующей, код на самом деле может выполняться в неочевидном порядке.
Всё зависит от времени завершения конкрентных задач.

Асинхронность важна во многих приложениях.
Так, игра на мобильном телефоне должна давать вам как пользователю возможность выйти из приложения, даже если в этот момент вы управляете бегающей сосиской.
Или при скачивании чего-либо на компьютере: экран остаётся кликабельным, и вы даже можете прервать загрузку.
Упомянутые процессы не блокируют работу: приложение всегда способно обрабатывать другие действия пользователя.
Или при работе сервера (например почтового, Gmail) сервер никогда целиком не переключается на работу с конкретным пользователем: только при попытке выполнить какое-то действие со стороны пользователя (например при отправке сообщения) сервер откликнется на запрос, но при бездействии со стороны пользователя (а пользователь, как правило, большую часть времени бездействует) сервер переключится на работу с кем-то ещё.

Преимущество же синхронного кода по сравнению с асинхронным — в простоте разработки, в понятности и читаемости кода.
Писать асинхронный код тяжелее, так как это требует более глубокого понимания происходящих внутри процессов и того, как они будут взаимодействовать между собой.

In [1]:
import asyncio
import datetime
import time

from typing import Awaitable

# Kostyl' to run a new loop in Jupyter Notebook
# (we can do without it;
#  still, for demonstration purposes, we are going to create a loop a couple of times)
import nest_asyncio

from aiofile import async_open

# https://github.com/ipython/ipython/issues/11338
# https://stackoverflow.com/a/56434301/8094251
nest_asyncio.apply()

In [11]:
LITTLE_TIME_INTERVAL = 0.1
BIGGER_TIME_INTERVAL = 0.5

### Sync, or "How Normal People Do It"<a id="sync"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

In [12]:
def print_hello() -> None:
    time.sleep(BIGGER_TIME_INTERVAL)
    
    print('Hello')

def print_world() -> None:
    time.sleep(LITTLE_TIME_INTERVAL)
    
    print('World')

In [13]:
print_hello()
print_world()

Hello
World


In [14]:
del print_hello
del print_world

### Async, But... ("Old-Way Acync")<a id="async-but"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

In [15]:
def print_hello():
    while True:
        time.sleep(BIGGER_TIME_INTERVAL)

        print('Hello')
        
        yield None


def print_world():
    while True:
        time.sleep(LITTLE_TIME_INTERVAL)

        print('World')
        
        yield None

In [16]:
coro1 = print_hello()
coro2 = print_world()

In [17]:
next(coro1)
next(coro2)

Hello
World


In [18]:
del print_hello
del print_world
del coro1, coro2

### Kinda Async, or "Looks Like Asyncio but Doesn't Quack Like Asyncio"<a id="kinda-async"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Просто `async def` ещё не делает метод асинхронным:

In [19]:
async def print_hello_async() -> Awaitable[None]:
    time.sleep(BIGGER_TIME_INTERVAL)
    
    print('Hello')

async def print_world_async() -> Awaitable[None]:
    time.sleep(LITTLE_TIME_INTERVAL)
    
    print('World')

Корутина, но не совсем:

In [20]:
coro = print_hello_async()

In [21]:
coro

<coroutine object print_hello_async at 0x00000182AEDF2E08>

In [22]:
try:
    next(coro)
except TypeError as error:
    print(error)

'coroutine' object is not an iterator


In [23]:
future1 = asyncio.ensure_future(print_hello_async())
future2 = asyncio.ensure_future(print_world_async())

Hello
World


In [24]:
del print_hello_async
del print_world_async
del future1, future2

Даже если просто вставить внутрь метода "что-то из asyncio", то всё равно не будет асинхронно:

In [25]:
async def print_hello_async() -> Awaitable[None]:
    asyncio.sleep(BIGGER_TIME_INTERVAL)
    
    print('Hello')


async def print_world_async() -> Awaitable[None]:
    asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    print('World')

In [26]:
future1 = asyncio.ensure_future(print_hello_async())
future2 = asyncio.ensure_future(print_world_async())

Hello
World


> coroutine 'sleep' was never awaited...

🤔

In [27]:
del print_hello_async
del print_world_async
del future1, future2

### Real Async, or "Finally Something Strange is Starting to Happen"<a id="real-async"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

In [28]:
async def print_hello_async() -> Awaitable[None]:
    await asyncio.sleep(BIGGER_TIME_INTERVAL)
    
    print('Hello')


async def print_world_async() -> Awaitable[None]:
    await asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    print('World')

In [29]:
future1 = asyncio.ensure_future(print_hello_async())
future2 = asyncio.ensure_future(print_world_async())

World
Hello


In [30]:
del print_hello_async
del print_world_async
del future1, future2

#### ...Even This Way

Даже если не ждать, Event loop всё равно переключает:

In [31]:
async def print_hello_async() -> Awaitable[None]:
    await asyncio.sleep(0)  # No actual sleep!
    
    print('Hello')


async def print_world_async() -> Awaitable[None]:
    print('World')

In [32]:
future1 = asyncio.ensure_future(print_hello_async())
future2 = asyncio.ensure_future(print_world_async())

World
Hello


In [33]:
del print_hello_async
del print_world_async
del future1, future2

#### Many Switches

In [34]:
async def print_hello_world_async() -> Awaitable[None]:
    print('Hello')
    
    await asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    print('world!')

Запустим несколько одинаковых задач асинхронно.
Видно, что никакая задача не блокирует поток:

In [35]:
_ = asyncio.ensure_future(print_hello_world_async())
_ = asyncio.ensure_future(print_hello_world_async())
_ = asyncio.ensure_future(print_hello_world_async())

Hello
Hello
Hello
world!
world!
world!


In [36]:
del print_hello_world_async

#### "Complex" Async (Async from Async)

И ещё один вариант асинхронного вывода строки `"Hello world!"` (источник: http://larsyencken.github.io/gitbook-mcpy/async/asyncio.html): выводится `"Hello"`, а в это же время "выполняется" функция, возвращающая строку `"world!"`.
Как только второе слово готово, оно тоже выводится.

In [37]:
async def print_hello_world_async() -> Awaitable[None]:
    world = asyncio.ensure_future(get_world_async())
    
    await asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    print('Hello', end=' ')
    print(await world)


async def get_world_async() -> Awaitable[str]:
    await asyncio.sleep(BIGGER_TIME_INTERVAL)
    
    return 'World'

In [38]:
_ = asyncio.ensure_future(print_hello_world_async())

Hello World


In [39]:
del print_hello_world_async
del get_world_async

### Task, or Something That Is Not Ready Yet<a id="task"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

...but will be ready sometime in the future.

In [59]:
async def get_slow_hello_async() -> Awaitable[str]:
    await asyncio.sleep(BIGGER_TIME_INTERVAL)
    
    return 'Hello world!'

In [61]:
future = asyncio.ensure_future(get_slow_hello_async())

print(f'Future: {future}')
print(f'Done: {future.done()}')

future.cancel()

Future: <Task pending coro=<get_slow_hello_async() running at <ipython-input-59-6c466166ddae>:1>>
Done: False


True

In [58]:
future.done()

True

In [62]:
future = asyncio.ensure_future(get_slow_hello_async())

# Jupyter Notebook allows to "wait for free"
result = await future

In [63]:
future.done()

True

In [64]:
future.result()

'Hello world!'

#### Run with Timeout

In [65]:
future = asyncio.ensure_future(
    asyncio.wait_for(get_slow_hello_async(), timeout=LITTLE_TIME_INTERVAL)
)

try:
    await future
except:  # TODO: Kostyl' to catch exception from async task (Python 3.6.7, asyncio==3.4.3, nest-asyncio==1.4.2)
    pass

In [66]:
future.done()

True

In [67]:
future

<Task finished coro=<wait_for() done, defined at D:\Maths\Anaconda3\lib\asyncio\tasks.py:321> exception=TimeoutError()>

In [68]:
future.exception()

concurrent.futures._base.TimeoutError()

#### Run Several Tasks

In [71]:
async def get_hello_async() -> Awaitable[str]:
    await asyncio.sleep(BIGGER_TIME_INTERVAL)
    
    return 'Hello'


async def get_world_async() -> Awaitable[str]:
    await asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    return 'World!'


async def _main() -> Awaitable[None]:
    done, pending = await asyncio.wait(
        [get_hello_async(), get_world_async()], return_when='FIRST_COMPLETED'
    )
    
    print(f'Done tasks: {done}.')
    print(f'Pending tasks: {pending}.')

In [72]:
future = asyncio.ensure_future(_main())

Done tasks: {<Task finished coro=<get_world_async() done, defined at <ipython-input-71-cb3403a0a25e>:7> result='World!'>}.
Pending tasks: {<Task pending coro=<get_hello_async() running at <ipython-input-71-cb3403a0a25e>:2> wait_for=<Future pending cb=[Task._wakeup()]>>}.


#### "Bad" Task

Сделаем что-нибудь... "плохое"!

In [73]:
async def do_something_bad_async() -> Awaitable[None]:
    await asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    1 / 0

In [74]:
task = asyncio.ensure_future(do_something_bad_async())

 Всё ОК?!

In [75]:
task

<Task finished coro=<do_something_bad_async() done, defined at <ipython-input-73-d99b5e225bc7>:1> exception=ZeroDivisionError('division by zero',)>

In [76]:
try:
    task.result()
except ZeroDivisionError as error:
    print(error)

division by zero


Объект исключения:

In [77]:
task.exception()

ZeroDivisionError('division by zero')

В общем, с задачами надо осторожно) Они могут завершаться "не удачно", но этого сходу не видно.

"When we suffer, we do it in silence."

#### Callback: Do This After The Task

In [78]:
async def get_world_async() -> Awaitable[str]:
    await asyncio.sleep(BIGGER_TIME_INTERVAL)
    
    return 'World!'

def hello_callback(task) -> None:
    print(f'Hello {task.result()}')

In [79]:
future = asyncio.ensure_future(get_world_async())
future.add_done_callback(hello_callback)

Hello World!


#### Coroutine vs. Future vs. Task

"В целом одно и то же, но под разными именами".

In [80]:
async def print_hello_async() -> Awaitable[str]:
    await asyncio.sleep(LITTLE_TIME_INTERVAL)
    
    message = 'Hello'
    
    print(message)
    
    return message


async def print_world_async() -> Awaitable[str]:
    await asyncio.sleep(0.1)

    message = 'World'
    
    print(message)
    
    return message

**Coroutines**

In [81]:
coroutines = [
    print_hello_async(),
    print_world_async(),
]

In [82]:
loop = asyncio.get_event_loop()

loop.run_until_complete(
    asyncio.gather(*coroutines)
)

Hello
World


['Hello', 'World']

In [83]:
del coroutines

**Futures**

In [84]:
coroutines = [
    print_hello_async(),
    print_world_async(),
]

# Registering tasks in the Event Loop:
futures = [
    asyncio.ensure_future(coroutine)
    for coroutine in coroutines
]

Hello
World


In [85]:
del coroutines
del futures

**Tasks**

Some task-related methods are available only starting from Python 3.7

In [86]:
loop = asyncio.get_event_loop()

try:
    _ = asyncio.create_task(print_hello_async())
    _ = asyncio.create_task(print_world_async())
except AttributeError:
    print("Sorry, your Python is probably too old :(")

Sorry, your Python is probably too old :(


Ладно, это было совсем по-игрушечному.
Пора посмотреть что-то поинтереснее...

## "Real-World" Example \#1: Books<a id="books"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Чтение из файла — пример I/O операции.
Значит, можно пытаться ускорить с помощью `asyncio`.

Использующиеся далее книги:
* [Толстой Лев - Война и мир. Книга 1 (том 1 и том 2)](https://royallib.com/book/tolstoy_lev/voyna_i_mir_kniga_1.html)
* [Толстой Лев - Война и мир. Книга 1 (том 3 и том 4)](https://royallib.com/book/tolstoy_lev/voyna_i_mir_kniga_2.html)

In [92]:
NUM_TIMES = 100

BOOK1 = 'War and Peace. Book 1.txt'
BOOK2 = 'War and Peace. Book 2.txt'

In [93]:
def _main():
    for _ in range(NUM_TIMES):
        with open(BOOK1, 'r', encoding='utf8') as file:
            file.read()

        with open(BOOK2, 'r', encoding='utf8') as file:
            file.read()

In [94]:
start = time.time()

_main()

finish = time.time()
    
print(f'Elapsed time: {finish - start}.')

Elapsed time: 3.0557446479797363.


In [95]:
async def _main_async():
    for _ in range(NUM_TIMES):
        async with async_open(BOOK1, 'r', encoding='utf8') as file:
            await file.read()

        async with async_open(BOOK2, 'r', encoding='utf8') as file:
            await file.read()

In [96]:
start = time.time()

asyncio.run(_main_async())

finish = time.time()
    
print(f'Elapsed time: {finish - start}.')

Elapsed time: 2.6573545932769775.


Но не стоит "увлекаться". Создание задач — дополнительная нагрузка. Надо понимать, в какой момент связанные с `asyncio` методы создают задачи. И если этих задач создаётся очень много, а действие в каждой задаче небольшое, то, скорее всего, с `asyncio` станет только хуже.

In [97]:
async def _not_so_good_main_async():
    for _ in range(NUM_TIMES):
        async with async_open(BOOK1, 'r', encoding='utf8') as file:
            async for line in file:
                pass

        async with async_open(BOOK2, 'r', encoding='utf8') as file:
            async for line in file:
                pass

In [98]:
timeout = 10
start = time.time()

try:
    asyncio.run(
        asyncio.wait_for(_not_so_good_main_async(), timeout=timeout)
    )
except:  # TODO: kostyl' to pass TimeoutError
    pass

finish = time.time()
elapsed_time = finish - start

print(f"Elapsed time: {elapsed_time if elapsed_time < 0.9 * timeout else 'infty'}.")

Elapsed time: infty.


## "Real-World" Example \#2: Butterbrot<a id="butterbrot"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

<br>

<figure>
    <img src="./images/poirot3_v3-2.jpg" width="80%">
    <figcaption style="text-align: center">
        <a href="https://dzen.ru/media/art_deco/kino-i-ar-deko-angliiskii-zavtrak-s-erkiulem-puaro-5f2ef64d3eeb070272878f38">
            <em>dzen.ru/media/art_deco/kino-i-ar-deko-angliiskii-zavtrak-s-erkiulem-puaro</em>.
        </a>
    </figcaption>
</figure>

Далее идёт переведённый с C# пример синхронного и асинхронного приготовления бутерброда.
Оригинал доступен по ссылке: https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/async.

Ниже определяется пайплайн приготовления завтрака `create_breakfast`.
Классы для составляющих завтрак пустые, так как они важны только в плане идеи.
Функция `sleep` представляет собой симуляцию затрат времени на приготовление.

Сначала рассмотрим последовательное приготовление ("засунули хлеб в тостер, стоим и ждём").

In [99]:
def create_breakfast():
    cup = pour_coffee()
    print("Coffee is ready")

    eggs = fry_eggs(2)
    print("Eggs are ready")

    bacon = fry_bacon(3)
    print("Bacon is ready")

    toast = toast_bread(2)
    apply_butter(toast)
    apply_jam(toast)
    print("Toast is ready")

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


class Coffee:
    pass

class Egg:
    pass

class Bacon:
    pass

class Toast:
    pass

class Juice:
    pass

class OrangeJuice(Juice):
    pass


def sleep():
    num_seconds = 3

    time.sleep(num_seconds)


def pour_coffee() -> Coffee:
    print("Pouring coffee")

    return Coffee()


def fry_eggs(num_eggs: int) -> Egg:
    print("Warming the egg pan...")
    sleep()
    sleep()
    
    print(f"Cracking {num_eggs} eggs")
    
    print("Cooking the eggs ...")
    sleep()
    
    print("Putting eggs on plate")

    return Egg()


def fry_bacon(num_slices: int) -> Bacon:
    print(f"Putting {num_slices} slices of bacon in the pan")
    
    print("Cooking first side of bacon...")
    sleep()
    
    for _ in range(num_slices):
        print("Flipping a slice of bacon")

    print("Cooking the second side of bacon...")
    sleep()
    
    print("Putting bacon on plate")

    return Bacon()


def toast_bread(num_slices: int) -> Toast:
    for _ in range(num_slices):
        print("Putting a slice of bread in the toaster")

    print("Start toasting...")
    sleep()

    print("Remove toast from toaster")

    return Toast()


def apply_butter(toast: Toast) -> None:
    print("Putting butter on the toast")


def apply_jam(toast: Toast) -> None:
    print("Putting jam on the toast")

    
def pour_orange_juice() -> OrangeJuice:
    print("Pouring orange juice")

    return OrangeJuice()

Функция-измеритель времени, затрачиваемого на приготовление завтрака (время считается по результатам одного запуска).

In [100]:
def measure_time(func):
    start_time = datetime.datetime.now()
    
    func()
    
    finish_time = datetime.datetime.now()
    elapsed_time = finish_time - start_time
    
    print(f'\nElapsed time: {elapsed_time}')

Сколько времени занимает приготовление завтрака:

In [101]:
measure_time(
    create_breakfast
)

Pouring coffee
Coffee is ready
Warming the egg pan...
Cracking 2 eggs
Cooking the eggs ...
Putting eggs on plate
Eggs are ready
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Putting bacon on plate
Bacon is ready
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:18.050627


Теперь — асинхронное (типа) приготовление:

In [102]:
async def create_breakfast_async():
    cup = pour_coffee()
    print("Coffee is ready")

    eggs = await fry_eggs_async(2)
    print("Eggs are ready")

    bacon = await fry_bacon_async(3)
    print("Bacon is ready")

    toast = await toast_bread_async(2)
    apply_butter(toast)
    apply_jam(toast)
    print("Toast is ready")

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


async def sleep_async():
    num_seconds = 3

    await asyncio.sleep(num_seconds)


async def fry_eggs_async(num_eggs: int) -> Awaitable[Egg]:
    print("Warming the egg pan...")
    await sleep_async()
    await sleep_async()
    
    print(f"Cracking {num_eggs} eggs")
    
    print("Cooking the eggs ...")
    await sleep_async()
    
    print("Putting eggs on plate")

    return Egg()


async def fry_bacon_async(num_slices: int) -> Awaitable[Bacon]:
    print(f"Putting {num_slices} slices of bacon in the pan")
    
    print("Cooking first side of bacon...")
    await sleep_async()
    
    for _ in range(num_slices):
        print("Flipping a slice of bacon")

    print("Cooking the second side of bacon...")
    await sleep_async()
    
    print("Putting bacon on plate")

    return Bacon()


async def toast_bread_async(num_slices: int) -> Awaitable[Toast]:
    for _ in range(num_slices):
        print("Putting a slice of bread in the toaster")

    print("Start toasting...")
    await sleep_async()

    print("Remove toast from toaster")

    return Toast()

Смотрим время...

In [103]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Cracking 2 eggs
Cooking the eggs ...
Putting eggs on plate
Eggs are ready
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Putting bacon on plate
Bacon is ready
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:18.057758


Время не изменилось!
Дело в том, что просто вставки `async` и `await` в исходный код ничего фактически не меняют.
Надо "включить асинхронную логику".
Ниже мы это и сделаем!
Надо создать задачи, но ждать их завершения сразу не надо: пусть играют на фоне, а дождаться их мы должны только тогда, когда нам нужен "осязаемый" результат.

In [104]:
async def create_breakfast_v2_async():
    """Concurrent breakfast
    """
    cup = pour_coffee()
    print("Coffee is ready")

    eggs_task = asyncio.ensure_future(
        fry_eggs_async(2)
    )
    bacon_task = asyncio.ensure_future(
        fry_bacon_async(3)
    )
    toast_task = asyncio.ensure_future(
        toast_bread_async(2)
    )
    
    toast = await toast_task
    apply_butter(toast)
    apply_jam(toast)
    print("Toast is ready")
    
    eggs = await eggs_task
    print("Eggs are ready")
    
    bacon = await bacon_task
    print("Bacon is ready")

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")

И время...

In [105]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_v2_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Cracking 2 eggs
Cooking the eggs ...
Putting bacon on plate
Putting eggs on plate
Eggs are ready
Bacon is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:09.021512


...уменьшилось! Ура!

Но это не предел улучшениям: пара синхронных одномоментных действий (намазывание масла и джема на хлеб) завязаны на одно асинхронное (приготовление хлеба в тостере).
Итого все эти действия вместе представляют собой асинхронный блок.
Давайте объединим всё, связанное с хлебом, в один асинхронный блок!

In [106]:
async def create_breakfast_v3_async():
    """Concurrent and recomposed breakfast
    """
    cup = pour_coffee()
    print("Coffee is ready")

    eggs_task = asyncio.ensure_future(
        fry_eggs_async(2)
    )
    bacon_task = asyncio.ensure_future(
        fry_bacon_async(3)
    )
    toast_task = asyncio.ensure_future(
        toast_bread_with_butter_and_jam_async(2)
    )
    
    eggs = await eggs_task
    print("Eggs are ready")
    
    bacon = await bacon_task
    print("Bacon is ready")

    toast = await toast_task
    print("Toast is ready")
    
    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


async def toast_bread_with_butter_and_jam_async(num_slices: int) -> Toast:
    toast = await toast_bread_async(num_slices)
    
    apply_butter(toast)
    apply_jam(toast)

    return toast

Есть ли улучшения?..

In [107]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_v3_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Cracking 2 eggs
Cooking the eggs ...
Putting bacon on plate
Putting eggs on plate
Eggs are ready
Bacon is ready
Toast is ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:09.025049


Увы, в данном случае они слишком малы, чтобы мы могли увидеть разницу.
Но это не значит, что программу не стоит продолжать улучшать :)
Ещё один важный пункт, достойный улучшения — ожидание выполнения задач.
Раньше мы ждали их в произвольном порядке (просто, как вздумалось).
Есть лучший способ — использовать функционал `asyncio` для получения задач просто по мере готовности (то есть порядок выполнения задач определяется просто с помощью Event Loop-а).
Ниже — финальный вариант приготовления завтрака:

In [108]:
async def create_breakfast_v5_async():
    """Concurrent and recomposed breakfast, with improved await
    """
    cup = pour_coffee()
    print("Coffee is ready")

    loop = asyncio.get_event_loop()

    eggs_task = asyncio.ensure_future(
        fry_eggs_async(2)
    )
    bacon_task = asyncio.ensure_future(
        fry_bacon_async(3)
    )
    toast_task = asyncio.ensure_future(
        toast_bread_with_butter_and_jam_async(2)
    )
    
    tasks = [eggs_task, bacon_task, toast_task]
    tiny_timeout = 0.01
        
    while len(tasks) > 0:
        finished_tasks, unfinished_tasks = loop.run_until_complete(
            asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
        )

        for task in finished_tasks:
            assert task.done()

            if task is eggs_task:
                print("Eggs are ready")
            elif task is bacon_task:
                print("Bacon is ready")
            elif task is toast_task:
                print("Toast is ready")

            tasks.remove(task)
    
    eggs = eggs_task.result()
    bacon = bacon_task.result()
    toast = toast_task.result()

    orange_juice = pour_orange_juice()
    print("Orange juice is ready")
    
    print("Breakfast is ready!")


async def toast_bread_with_butter_and_jam_async(num_slices: int) -> Toast:
    toast = await toast_bread_async(num_slices)
    
    apply_butter(toast)
    apply_jam(toast)

    return toast

И время в лучшем случае...

In [109]:
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        create_breakfast_v5_async()
    )

measure_time(main)

del main

Pouring coffee
Coffee is ready
Warming the egg pan...
Putting 3 slices of bacon in the pan
Cooking first side of bacon...
Putting a slice of bread in the toaster
Putting a slice of bread in the toaster
Start toasting...
Flipping a slice of bacon
Flipping a slice of bacon
Flipping a slice of bacon
Cooking the second side of bacon...
Remove toast from toaster
Putting butter on the toast
Putting jam on the toast
Toast is ready
Cracking 2 eggs
Cooking the eggs ...
Putting bacon on plate
Bacon is ready
Putting eggs on plate
Eggs are ready
Pouring orange juice
Orange juice is ready
Breakfast is ready!

Elapsed time: 0:00:09.033717


...изменилось не сильно, по сравнению с предыдущими асинхронными выполнениями.
Увы, улучшения — а они должны быть :) — слишком малы, чтобы их можно было увидеть за один прогон программы.
В оригинальном примере время ожидания измерялось десятками минут (хотя в примерах кода стоят ожидания в $3$ секунды, как в ноутбуке).
Столько ждать не хочется) хотя интересно бы было проверить... но, кажется, разница между async реализациями не должна зависеть от времени одного `sleep`?.. 🤔

## References<a id="references"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

### "Main"

* https://youtu.be/kdzL3r-yJZY

Получасовая видеолекция про асинхронность в целом.
И там тоже есть пример с приготовлением еды :)

* https://realpython.com/python-concurrency/

Про корутины + нити + параллельность, с примерами.

* https://ru.stackoverflow.com/a/902762/252621

В двух словах (на русском) про то, что такое `Future` и `Task`.

* https://channel9.msdn.com/Series/Three-Essential-Tips-for-Async/Three-Essential-Tips-For-Async-Introduction

Несколько видео про C#, но в самом первом видео понятно и общо рассказывают про суть Async (видео на пять минут).

* https://tech.chefclub.tv/an-asyncio-io-bound-producer-playing-with-a-cpu-bound-consumer

Пример по `asyncio` про consumer + producer (скачивание видео файлов и их обработка).

* http://www.blog.pythonlibrary.org/2016/07/26/python-3-an-intro-to-asyncio/

Тоже в целом про `asyncio`, тоже с примером (только скачивание файлов).


### Also Noteworthy

* https://stackoverflow.com/a/34101652/8094251

Объясняют разницу между `yield` из генератора и `send` в корутину (что в каком порядке там происходит).

* https://stackoverflow.com/a/39792393/8094251

Общий комментарий про то, что такое корутины.

* https://stackoverflow.com/questions/59260268/is-there-ever-a-reason-to-return-await-in-python-asyncio

Заметка про `return await`.

* https://habr.com/ru/post/314606/

Пара заметок про `asyncio` из опыта разработчика в продакшене.
Есть слова о правильном закрытии Event Loop-а (о том, что его надо закрывать).


### "Advanced"

* https://stackoverflow.com/a/51177895/8094251

"Мозговыносящий" комментарий-статья на StackOverflow.
В первой части объясняются идейные отличия нитей, процессов и задач.
Есть несложный пример кастомного Event Loop (!)

* https://softwareengineering.stackexchange.com/a/352263/348557

Небольшой экскурс в историю корутин (в разных языках программирования).

* https://www.python.org/dev/peps/pep-0492/

PEP 492 — Coroutines with async and await syntax.
Про то, как это всё происходило в Питоне.


### Probably Also Noteworthy

* https://realpython.com/python-gil/

Про [GIL](https://ru.wikipedia.org/wiki/Global_Interpreter_Lock).

* [David M. Beazley 1](http://www.dabeaz.com/generators/)

Generator Tricks for Systems Programmers, 2008.

* [David M. Beazley 2](http://dabeaz.com/coroutines/)

A Curious Course on Coroutines and Concurrency, 2009.

Презентация.
"Живая" (с юмором и т.д.).
Есть пример про смешивание генерации и принятие значений в корутине.

* [David M. Beazley 3](http://www.dabeaz.com/finalgenerator/)

Generators: The Final Frontier, 2014.

* https://realpython.com/async-io-python/

Подробнее про `asyncio`, в начале тоже есть слова про отличия подходов к параллельности в Питоне.